In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
dados = pd.read_csv('../input/ebay-reviews/ebay_reviews.csv')
dados = dados.drop(['category'], axis=1)
#Troca-se as notas pela qualidade de negativa ou positiva da review.
dados["rating"]  = dados["rating"].replace([1,2,3,4,5], ["Negativa", "Negativa", "Positiva", "Positiva", "Positiva"])

In [ ]:
Vectorizer = CountVectorizer()
Model = LogisticRegression(max_iter = 4000)

#junta-se as informações do título e do conteúdo da review em uma só variável
x = dados['review title'] + " " + dados['review content']
y = dados['rating']
y.value_counts()

In [ ]:
#Há mais de 42k reviews positivas e apenas 1942 negativas.
#O under sampling balanceia esses dados, pegando 1942 amostras (samples) das 42k positivas.
redutor = RandomUnderSampler()
y = np.array(y)
x = np.array(x)
y = np.reshape(y, (-1,1))
x = np.reshape(x, (-1,1))
x_r, y_r = redutor.fit_resample(x, y)
print(len(y_r))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_r,y_r)
x_test = x_test.reshape(-1)
x_train = Vectorizer.fit_transform(x_train.reshape(-1))
x_test = Vectorizer.transform(x_test)
Model.fit(x_train, y_train)
result = Model.predict(x_test)
print(confusion_matrix(y_test, result))
accuracy_do_modelo = accuracy_score(y_test, result)
print(accuracy_do_modelo)

In [ ]:
predicao1 = Model.predict(Vectorizer.transform(["Excellent product!"]))
predicao2 = Model.predict(Vectorizer.transform(["This is probably the best thing i have ever bought"]))
predicao3 = Model.predict(Vectorizer.transform(["It works fine! Thank you!"]))
predicao4 = Model.predict(Vectorizer.transform(["Do not buy it. Horrible product"]))
predicao5 = Model.predict(Vectorizer.transform(["My delivery is 2 months delayed. Pure lack of profissionalism"]))
predicao6 = Model.predict(Vectorizer.transform(["Many disadvantages, i do not recommend it"]))

print(predicao1, predicao2, predicao3, predicao4, predicao5, predicao6)

Excellent product!
This is probably the best thing i have ever bought
It works fine! Thank you!



In [ ]:
#O modelo funciona, mas como uma pessoa de fora da computação poderia utilizar isso?
n, pos, neg = int(input("Número de reviews a serem analisadas:")), 0, 0
for x in range(1, n+1):
    predicoes = []
    review = input(f"Review {x}:")
    resultado = Model.predict(Vectorizer.transform([review]))
    if resultado[0] == "Positiva":
        pos += 1
    else:
        neg += 1
plt.figure(figsize=(5,5))
plt.title("Análise das reviews:")
plt.pie(x=[pos, neg], labels=["Reviews positivas", "Reviews Negativas"],autopct = '%1.2f%%', startangle=90, shadow=True)
print('')
plt.show()
print(f"Reviews positivas: {pos} \nReviews negativas: {neg} \nObservação: as predições apresentadas contam com uma exatidão de {accuracy_do_modelo:.1%}")